In [25]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import tensorflow as tf

2024-10-13 10:13:26.626413: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
CONFIGURATION = {
    "BATCH_SIZE": 32,
    "IM_SIZE": 256,
    "LEARNING_RATE": 1e-3,
    "N_EPOCHS": 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASSES": 3,
    "PATCH_SIZE": 16,
    "PROJ_DIM": 768,
    "CLASS_NAMES": ["angry", "happy", "sad"],
}

In [4]:
df = pd.read_csv("./Handwritten_Signature_Dataset/file_identity.csv")

In [5]:
df.head()

,FILENAME,IDENTITY,VIETNAMESE IDENTITY
0,FILE_1.png,BUI DUONG QUOC THAI,BÙI DƯƠNG QUỐC THÁI
1,FILE_2.png,CUONG,CƯỜNG
2,FILE_3.png,DANG NGOC TRAM ANH,ĐẶNG NGỌC TRÂM ANH
3,FILE_4.png,DINH PHUONG TIN,ĐINH PHƯƠNG TÍN
4,FILE_5.png,DINH THI TUYET NHUNG,ĐINH THỊ TUYẾT NHUNG


In [ ]:
plt.figure(figsize=(8, 8))
for i in range(6):
    plt.subplot(3, 2, i+1)
    dir = "./Handwritten_Signature_Dataset/train/" + df.loc[i, "FILENAME"]
    img = cv2.imread(dir)
    plt.title(df.loc[i, "VIETNAMESE IDENTITY"])
    plt.imshow(img)
    plt.subplots_adjust(hspace=1)

In [41]:
IM_SIZE = 224
@tf.function
def resizing_rescale(image):
    image = tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.
    image = tf.image.rgb_to_grayscale(image)
    return image

In [ ]:
train_dataset = []
for i in range(len(df)):
    dir = "./Handwritten_Signature_Dataset/train/" + df.loc[i, "FILENAME"]
    img = cv2.imread(dir, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    train_dataset.append(img.reshape(-1))

In [50]:
lengths = df.map(len)
max = df.loc[lengths.idxmax().max()]
max_str = max.name
print(len(max.IDENTITY))

21


In [51]:
alphabets = u"ABCDEFGHIJKLMNOPQRSTUVWXYZ-' "
max_str_len = len(max.IDENTITY)
num_of_char = len(alphabets) + 1
num_of_timestamps = 64

In [59]:
def label_to_num(label):
    label_num = []
    for ch in label:
        label_num.append(alphabets.find(ch))

    return label_num

def num_to_label(num):
    ret = ""
    for ch in num:
        if ch == -1:  # sCTC Blank
            break
        else:
            ret+=alphabets[ch]
    return ret

In [73]:
train_y_1 = np.ones([len(df), max_str_len]) * -1
train_label_len = np.zeros([len(df), 1])
train_input_len = np.ones([len(df), 1]) * (num_of_timestamps-2)
train_output = np.zeros([len(df)])

for i in range(len(df)):
    train_label_len[i] = len(df.loc[i, 'IDENTITY'])
    train_y_1[i, 0:len(df.loc[i, 'IDENTITY'])]= label_to_num(df.loc[i, 'IDENTITY'])

In [75]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR 

In [82]:
svr = SVR()
multi_target = MultiOutputRegressor(svr)
multi_target.fit(train_dataset, train_y_1)

MultiOutputRegressor(estimator=SVR())

In [77]:
multi_target.score(train_dataset, train_y_1)

0.05673560012304625